In [ ]:
import os
import glob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xa
from cartopy.crs import PlateCarree, LambertAzimuthalEqualArea
from cartopy.feature import LAND

In [ ]:
laea = LambertAzimuthalEqualArea(central_latitude=90, central_longitude=0)
pc = PlateCarree()
buoys = pd.read_feather('../iabp_validation/data/segments.feather').dropna()


In [ ]:
x, y = laea.transform_points(pc, buoys['Lon'].values, buoys['Lat'].values).T[:2]

In [ ]:
minx, maxx = -2.1e6, 2e6
miny, maxy = -2.4e6, 3e6
res = 25000
width = int((maxx - minx) / res)
height = int((maxy - miny) / res)
col = np.clip(((x - minx) / res).astype(int), 0, width - 1)
row = np.clip(((y - miny) / res).astype(int), 0, height - 1)
# create absolute index from col and row
abs_index = row * width + col
count_matrix = np.zeros((height, width))
mean_matrix = np.zeros((height, width))
np.add.at(count_matrix, (row, col), 1)
np.add.at(mean_matrix, (row, col), buoys.Age)
count_matrix[count_matrix < 1] = np.nan
mean_matrix = mean_matrix / count_matrix / 365

fig, axs = plt.subplots(1, 2, figsize=(10,7),subplot_kw={'projection': laea})
imsh0 = axs[0].imshow(count_matrix, cmap='viridis', clim=[0, 200], extent=[minx, maxx, miny, maxy], origin='lower', interpolation='nearest')
plt.colorbar(imsh0, ax=axs[0], orientation='vertical', pad=0.05, shrink=0.5)
imsh1 = axs[1].imshow(mean_matrix, cmap='viridis', clim=[0, 2], extent=[minx, maxx, miny, maxy], origin='lower', interpolation='nearest')
plt.colorbar(imsh1, ax=axs[1], orientation='vertical', pad=0.05, shrink=0.5)
axs[0].coastlines()
axs[0].add_feature(LAND, zorder=10)
axs[1].coastlines()
axs[1].add_feature(LAND, zorder=10)

axs[0].set_title('Num. IABP observations')
axs[1].set_title('Mean IABP buoy age, years')
plt.tight_layout()
plt.savefig('figure01_iabp_buoys.png', bbox_inches='tight', dpi=100, pad_inches=0.1)
plt.show()
